In [82]:
import json

# JSON 파일 로드
file_path = "../데이터/highschool_english_dataset.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 질문을 리스트로 저장
questions = []
for key, value in data.items():
    if "questions" in value and "cleaned_text" in value["questions"]:
        questions.append(value["questions"]["cleaned_text"])

print(f"총 {len(questions)}개의 문제를 로드했습니다.")

총 14개의 문제를 로드했습니다.


#  OpenAI API를 사용해 질문을 벡터로 변환

In [83]:
import openai

# OpenAI 임베딩 함수 (최신 API 적용)
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding  # 최신 API 방식 적용

# 모든 질문을 임베딩 변환
question_embeddings = [get_embedding(q) for q in questions]

print(f"총 {len(question_embeddings)}개의 질문을 벡터로 변환 완료!")

총 14개의 질문을 벡터로 변환 완료!


# FAISS 데이터베이스 생성

In [84]:
import faiss
import numpy as np

# 벡터 차원 설정 (text-embedding-ada-002는 1536차원)
embedding_dim = 1536

# FAISS 인덱스 생성 (L2 거리 기반)
index = faiss.IndexFlatL2(embedding_dim)

# 임베딩 데이터를 numpy 배열로 변환
question_vectors = np.array(question_embeddings).astype("float32")

# FAISS 데이터베이스에 추가
index.add(question_vectors)

print(f"벡터 데이터베이스에 {index.ntotal}개의 벡터 저장 완료!")

벡터 데이터베이스에 14개의 벡터 저장 완료!


# FAISS를 이용한 유사 질문 검색

In [85]:
def search_similar_questions(query, top_k=3):
    # 입력 질문을 벡터로 변환
    query_vector = np.array(get_embedding(query)).astype("float32").reshape(1, -1)

    # 가장 가까운 질문 검색
    distances, indices = index.search(query_vector, top_k)

    # 결과 출력
    print("\n[가장 유사한 질문들]")
    for i in range(top_k):
        idx = indices[0][i]
        print(f"{i+1}. {questions[idx]} (거리: {distances[0][i]:.4f})")

# 예제 검색 실행
user_query = "Many countries suffer from a lack of water."
search_similar_questions(user_query)


[가장 유사한 질문들]
1. 영 제 3 교시 [1~3] 밑줄 친 부분의 뜻으로 가장 적절한 것을 고르시오. 1. David put a lot of effort into the exam. ① 노력 ② 의미 ③ 조언 ④ 좌절 2. Many countries suffer from a lack of water. ① 절약하다 ② 이용하다 ③ 낭비하다 ④ 고통받다 3. A: Excuse me, can I have soup instead of salad? B: Sure. I will bring it in a minute. ① ～로써 ② ～대신에 ③ ～덕분에 ④ ～와 함께 4. 두 단어의 의미 관계가 나머지 셋과 다른 것은? ① joy－sadness ② furniture－sofa ③ subject－science ④ animal－chimpanzee 5. 다음 광고문에서 언급되지 않은 것은? Moonlight Tour of the Beautiful Palace! Opening: Tuesday to Friday 7 p.m.－9 p.m. Reservation: Book on our website one day before (www.moonlight.co.kr) Entrance Fee: Free for everyone ① 운영 시간 ② 예약 방법 ③ 준비물 ④ 입장료 [6~8] 빈칸에 공통으로 들어갈 말로 가장 적절한 것을 고르시오. 6. ◦Can you explain to use the copy machine? ◦I can’t understand he solved the problem. ① what ② that ③ who ④ how 7. ◦The flood had caused serious to the village. ◦Don’t the surface of the table with the pencil. ① damage ② gesture ③ comment ④ concern 8. ◦All the people focused her performance. ◦This island’

# 임베딩 벡터 직접 확인하기

In [89]:
# 첫 번째 질문의 임베딩 벡터 확인
print(f"임베딩 벡터 (앞 10개 값): {question_embeddings[0][:10]}")  # 일부만 출력
print(f"벡터 차원 수: {len(question_embeddings[0])}")  # 1536인지 확인

임베딩 벡터 (앞 10개 값): [-0.011448552832007408, -0.00758728664368391, 0.02577783912420273, -0.014538918621838093, -0.021679893136024475, 0.03180979937314987, -0.0036110614892095327, -0.005626223050057888, -0.02387087419629097, -0.008067408576607704]
벡터 차원 수: 1536


# FAISS 벡터 데이터베이스에 저장된 벡터 확인

In [90]:
# FAISS 데이터베이스에 저장된 첫 번째 벡터 가져오기
faiss_vector = np.zeros((1, 1536), dtype="float32")
index.reconstruct(0, faiss_vector[0])

print(f"FAISS에 저장된 첫 번째 벡터 (앞 10개 값): {faiss_vector[0][:10]}")
print(f"FAISS 벡터 차원 수: {len(faiss_vector[0])}")

FAISS에 저장된 첫 번째 벡터 (앞 10개 값): [-0.01144855 -0.00758729  0.02577784 -0.01453892 -0.02167989  0.0318098
 -0.00361106 -0.00562622 -0.02387087 -0.00806741]
FAISS 벡터 차원 수: 1536


# FAISS 벡터 데이터베이스 저장소 정보 확인

In [91]:
# 저장된 벡터 개수 확인
print(f"FAISS에 저장된 총 벡터 개수: {index.ntotal}")

FAISS에 저장된 총 벡터 개수: 14


# FAISS 인덱스를 저장하는 코드 추가

In [92]:
import faiss

# FAISS 인덱스 저장
faiss.write_index(index, "faiss_index.bin")
print("✅ FAISS 인덱스를 'faiss_index.bin' 파일로 저장 완료!")

✅ FAISS 인덱스를 'faiss_index.bin' 파일로 저장 완료!
